In [23]:
import os
import shutil
from pathlib import Path
from dotenv import load_dotenv
import dataset  # Make sure this is the correct module name
import tifffile as tiff

load_dotenv()

data_path = Path(os.getenv("prepare_dataset_folder"))
processed_path = Path(os.getenv("processed_images_folder"))
split = "train"  # Adjust as needed for 'val' or 'test'

# Ensure the processed data folder exists
processed_path.mkdir(parents=True, exist_ok=True)

# Instantiate the dataset
forest_sat_dataset = dataset.MineSATDataset(split=split, data_path=data_path)


# Iterate over all dataset indices
for index in range(len(forest_sat_dataset)):
    # Get the filepaths for the current index
    filepath = forest_sat_dataset.filepaths[index]
    
    # Extract the original directory name
    original_dir_name = Path(filepath).name
    
    # Get the processed images
    images_dict = forest_sat_dataset.get_transformed_images(index)  # Using the new method
    
    # Create a new directory path for the processed images
    processed_dir_path = processed_path / original_dir_name
    processed_dir_path.mkdir(parents=True, exist_ok=True)
    
    # Save each processed image in the new directory
    for img_type, image in images_dict.items():
        # Skip saving the mask if you only want the indices and RGB images
        if img_type == "Mask":
            continue

        # Define the filename, maintaining the original directory name
        filename = f"{original_dir_name}_{img_type}.tif"
        # Define the full path
        filepath = processed_dir_path / filename
        # Save the image
        tiff.imwrite(filepath, image) 


AttributeError: 'MineSATDataset' object has no attribute 'get_transformed_images'

In [20]:

import numpy as np
import csv

def save_numerical_values_to_csv(numerical_values, csv_filename):
    # Flatten the numerical values and prepare them for writing to a CSV file
    flattened_data = {}
    for key, value in numerical_values.items():
        if isinstance(value, np.ndarray):
            flattened_data[key] = value.flatten()
        else:
            flattened_data[key] = value

    # Find the longest array in the flattened data
    max_length = max(len(v) for v in flattened_data.values())

    # Ensure all arrays are the same length, filling with NaN where necessary
    for key in flattened_data:
        length = len(flattened_data[key])
        if length < max_length:
            flattened_data[key] = np.pad(flattened_data[key], (0, max_length - length), constant_values=np.nan)

    # Write to CSV
    with open(csv_filename, 'w', newline='') as csvfile:
        fieldnames = list(flattened_data.keys())
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(max_length):
            row = {key: flattened_data[key][i] for key in fieldnames}
            writer.writerow(row)


In [14]:
numerical_values = forest_sat_dataset.get_numerical_values(index)
print(numerical_values)
save_numerical_values_to_csv(numerical_values, "../data_csv/data.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/lilllmeng/PycharmProjects/Capstone/prepare_dataset/image_directory_20230813_100/B07.tif'